In [12]:
from ortools.linear_solver import pywraplp as mip
import numpy.linalg as la
import numpy as np

graph_3d = {}
graph_3d[0, 0, 1, 0] = 0.7
graph_3d[0, 0, 1, 1] = 0.6
graph_3d[0, 1, 1, 0] = 0.2
graph_3d[0, 1, 1, 1] = 0.8
graph_3d[0, 0, 2, 0] = 0.9
graph_3d[0, 0, 2, 1] = 0.55
graph_3d[0, 1, 2, 0] = 0.1
graph_3d[0, 1, 2, 1] = 0.9
graph_3d[1, 0, 2, 0] = 0.9
graph_3d[1, 0, 2, 1] = 0.51
graph_3d[1, 1, 2, 0] = 0.01
graph_3d[1, 1, 2, 1] = 0.99
n_frames = 3


solver = mip.Solver('t', mip.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

pids_per_frame = {}
Tau = {}
costs = {}
for (tA, pidA, tB, pidB), score in graph_3d.items():
    Tau[tA, pidA, tB, pidB] = solver.BoolVar('t[%i,%i,%i,%i]' % (tA, pidA, tB, pidB))
    costs[tA, pidA, tB, pidB] = np.log(score / (1-score))
    if tA not in pids_per_frame:
        pids_per_frame[tA] = set()
    pids_per_frame[tA].add(pidA)
    if tB not in pids_per_frame:
        pids_per_frame[tB] = set()
    pids_per_frame[tB].add(pidB)

Sum = solver.Sum(Tau[edge] * costs[edge] for edge in graph_3d.keys())

for t1 in range(n_frames - 1):
    for t2 in range(t1 + 1, n_frames):
        for pid1 in pids_per_frame[t1]:
            solver.Add(
                solver.Sum(Tau[t1, pid1, t2, pid2] for pid2 in pids_per_frame[t2]\
                           if (t1, pid1, t2, pid2) in Tau) <= 1
            )
            
solver.Maximize(Sum)
RESULT = solver.Solve()

for (t1, pid1, t2, pid2), v in Tau.items():
    assert t1 < t2
    print(str((t1, pid1)) + '--' + str((t2, pid2)) + ', ', v.solution_value())

(0, 0)--(1, 0),  1.0
(0, 0)--(1, 1),  0.0
(0, 1)--(1, 0),  0.0
(0, 1)--(1, 1),  1.0
(0, 0)--(2, 0),  1.0
(0, 0)--(2, 1),  0.0
(0, 1)--(2, 0),  0.0
(0, 1)--(2, 1),  1.0
(1, 0)--(2, 0),  1.0
(1, 0)--(2, 1),  0.0
(1, 1)--(2, 0),  0.0
(1, 1)--(2, 1),  1.0
